In [14]:
import os
import shutil

shutil.rmtree("src", ignore_errors=True)
shutil.rmtree("data", ignore_errors=True)

os.makedirs("src", exist_ok=True)
os.makedirs("data", exist_ok=True)

main_py = r"""
import sys
import os
ROOT_DIR = os.path.dirname(os.path.abspath(__file__))
if ROOT_DIR not in sys.path:
    sys.path.append(ROOT_DIR)
from src.menu import Menu
def main():
    menu = Menu()
    menu.run()
if __name__ == "__main__":
    main()
"""
with open("main.py", "w") as f:
    f.write(main_py)

with open("src/__init__.py", "w") as f:
    f.write("")

expense_py = r"""
class Expense:
    def __init__(self, amount, category, date, description):
        self.amount = float(amount)
        self.category = category
        self.date = date
        self.description = description
    def __str__(self):
        return f"{self.date} | {self.category}: ₹{self.amount} - {self.description}"
"""
with open("src/expense.py", "w") as f:
    f.write(expense_py)

file_manager_py = r"""
import csv
import os
import shutil
DATA_FILE = "data/expenses.csv"
BACKUP_FILE = "data/expenses_backup.csv"
class FileManager:
    @staticmethod
    def load_expenses():
        expenses = []
        if not os.path.exists(DATA_FILE):
            return expenses
        with open(DATA_FILE, "r") as file:
            reader = csv.DictReader(file)
            for row in reader:
                expenses.append(row)
        return expenses
    @staticmethod
    def save_expenses(expenses):
        with open(DATA_FILE, "w", newline="") as file:
            writer = csv.writer(file)
            writer.writerow(["Date", "Category", "Amount", "Description"])
            for exp in expenses:
                writer.writerow([exp["Date"], exp["Category"], exp["Amount"], exp["Description"]])
    @staticmethod
    def backup_data():
        if os.path.exists(DATA_FILE):
            shutil.copy(DATA_FILE, BACKUP_FILE)
            return True
        return False
    @staticmethod
    def restore_backup():
        if os.path.exists(BACKUP_FILE):
            shutil.copy(BACKUP_FILE, DATA_FILE)
            return True
        return False
"""
with open("src/file_manager.py", "w") as f:
    f.write(file_manager_py)

utils_py = r"""
def validate_amount(amount):
    try:
        return float(amount)
    except:
        return None
def validate_date(date):
    import re
    pattern = r"^\d{4}-\d{2}-\d{2}$"
    if not re.match(pattern, date):
        return None
    return date
"""
with open("src/utils.py", "w") as f:
    f.write(utils_py)

reports_py = r"""
from collections import defaultdict
class Reports:
    @staticmethod
    def total_expense(expenses):
        return sum(float(x["Amount"]) for x in expenses)
    @staticmethod
    def category_summary(expenses):
        summary = defaultdict(float)
        for exp in expenses:
            summary[exp["Category"]] += float(exp["Amount"])
        return dict(summary)
    @staticmethod
    def monthly_summary(expenses, month):
        month_exp = []
        for exp in expenses:
            if exp["Date"].startswith(month):
                month_exp.append(exp)
        return month_exp
"""
with open("src/reports.py", "w") as f:
    f.write(reports_py)

menu_py = r"""
from src.file_manager import FileManager
from src.utils import validate_amount, validate_date
from src.reports import Reports
import shutil
class Menu:
    def __init__(self):
        self.expenses = FileManager.load_expenses()
    def clear(self):
        print("\n" * 3)
    def pause(self):
        input("Press Enter to continue...")
    def show_menu(self):
        print("==========================================")
        print("         PERSONAL FINANCE MANAGER")
        print("==========================================")
        print("1. Add New Expense")
        print("2. View All Expenses")
        print("3. View Category-wise Summary")
        print("4. Generate Monthly Report")
        print("5. Backup Data")
        print("6. Restore Backup")
        print("7. Exit")
        return input("Enter your choice (1-7): ")
    def run(self):
        while True:
            choice = self.show_menu()
            if choice == "1":
                self.add_expense()
            elif choice == "2":
                self.view_expenses()
            elif choice == "3":
                self.category_summary()
            elif choice == "4":
                self.monthly_report()
            elif choice == "5":
                self.backup_data()
            elif choice == "6":
                self.restore_backup()
            elif choice == "7":
                print("Goodbye!")
                break
            else:
                print("Invalid choice!")
    def add_expense(self):
        amount = validate_amount(input("Enter amount: "))
        if amount is None:
            print("Invalid amount!")
            return
        category = input("Enter category: ")
        date = validate_date(input("Enter date (YYYY-MM-DD): "))
        if date is None:
            print("Invalid date format!")
            return
        description = input("Enter description: ")
        expense = {"Amount": amount, "Category": category, "Date": date, "Description": description}
        self.expenses.append(expense)
        FileManager.save_expenses(self.expenses)
        print("Expense added successfully!")
        self.pause()
    def view_expenses(self):
        print("\n--- ALL EXPENSES ---")
        for e in self.expenses:
            print(f"{e['Date']} | {e['Category']}: ₹{e['Amount']} - {e['Description']}")
        self.pause()
    def category_summary(self):
        summary = Reports.category_summary(self.expenses)
        print("\n--- CATEGORY SUMMARY ---")
        for k, v in summary.items():
            print(f"{k}: ₹{v}")
        self.pause()
    def monthly_report(self):
        month = input("Enter month (YYYY-MM): ")
        data = Reports.monthly_summary(self.expenses, month)
        print("\n--- MONTHLY REPORT ---")
        for e in data:
            print(f"{e['Date']} | {e['Category']}: ₹{e['Amount']} - {e['Description']}")
        self.pause()
    def backup_data(self):
        if FileManager.backup_data():
            print("Backup created successfully!")
        else:
            print("No data to backup!")
        self.pause()
    def restore_backup(self):
        if FileManager.restore_backup():
            print("Backup restored!")
            self.expenses = FileManager.load_expenses()
        else:
            print("No backup found!")
        self.pause()
"""
with open("src/menu.py", "w") as f:
    f.write(menu_py)

with open("data/expenses.csv", "w") as f:
    f.write("Date,Category,Amount,Description\n")

print("Project created. Run using:")
print("!python main.py")


Project created. Run using:
!python main.py


In [15]:
!python main.py



         PERSONAL FINANCE MANAGER
1. Add New Expense
2. View All Expenses
3. View Category-wise Summary
4. Generate Monthly Report
5. Backup Data
6. Restore Backup
7. Exit
Enter your choice (1-7): 1
Enter amount: 1500
Enter category: Food
Enter date (YYYY-MM-DD): 1999-03-31
Enter description: Grocery
Expense added successfully!
Press Enter to continue...7
         PERSONAL FINANCE MANAGER
1. Add New Expense
2. View All Expenses
3. View Category-wise Summary
4. Generate Monthly Report
5. Backup Data
6. Restore Backup
7. Exit
Enter your choice (1-7): 7
Goodbye!
